# Pet Breeds Example

[Source](https://github.com/fastai/fastbook/blob/master/05_pet_breeds.ipynb)

In [ ]:
import fastai
from fastai.vision.all import *
path = untar_data(URLs.PETS)

In [ ]:
Path.BASE_PATH = path

In [ ]:
path.ls()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
def get_dls(bs, workers=None):
    path = untar_data(URLs.PETS)
    #source = untar_data(path)
    if workers is None: workers = min(8, num_cpus())
    batch_tfms = [aug_transforms(size=224, min_scale=0.75)]
    dblock = DataBlock(blocks = (ImageBlock, CategoryBlock),
                       get_items=get_image_files, 
                       splitter=RandomSplitter(seed=42),
                       get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                       item_tfms=Resize(460),
                       batch_tfms=aug_transforms(size=224, min_scale=0.75))
    return dblock.dataloaders(path/"images", bs=bs, num_workers=workers)

In [ ]:
# define the main function as a Jupyter cell

gpu = None  # set GPU to None for CPU usage
bs = 64 # batch size
arch = resnet34 # model architechture
runs = 1 # epochs

# gpu = setup_distrib(gpu)
if gpu is not None: torch.cuda.set_device(gpu)

dls = get_dls(bs)

for run in range(runs):
    print(f'Run: {run}')

    learn = cnn_learner(dls, arch, metrics=error_rate).to_fp16()

    print("Training in CPU/GPU context")
    learn.fine_tune(2)

In [ ]:
# display results, e.g., confusion matrix
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()